# Project invoices
*KUBIK Aleksander - KOBANA Johan - JOUYIT Matthieu - Thomas BOULAINE - DIA4*


Our problem : How can we analyze and visualize an online store’s activity using an invoice dataset to extract key indicators that support data-driven decisions?



In [37]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

import plotly_express as px
import plotly.graph_objects as go

from mlxtend.frequent_patterns import fpgrowth, association_rules

from prophet import Prophet


In [38]:
def load_data(file_path):
    """
    Function for reading a CSV file
    Input: the path to the CSV file (string)    
    Output: a dataframe containing the loaded dataset
    """
    df = pd.read_csv(file_path)   # Read the CSV into a pandas DataFrame
    return df                     # Return the full dataset

### **load_data()** - Load CSV Files
**Purpose**: Read CSV files into pandas DataFrames.  
**Input**: File path (string)  
**Output**: DataFrame ready for analysis  
**Use case**: Initial data loading from `invoices.csv` and city reference file  

**Code Approach**:  
Uses `pd.read_csv()` to load a CSV file directly into memory. Simple wrapper function for reusability and consistency across all data imports.

In [39]:
def basic_info(data):
    """
    Function to display basic information about a dataframe
    Input: a dataframe
    Output: printed information (shape, columns, dtypes, missing values)
    """
    print("Shape:", data.shape)                     # Print number of rows and columns
    print("\nColumns:", data.columns.tolist())      # Print list of column names

    print("\nData types:")                          # Print data types of each column
    print(data.dtypes)

    print("\nMissing values per column:")           # Print missing values for each column
    print(data.isna().sum())

    print("\nDescriptive statistics:")
    categorical_cols = [
    'first_name', 'city', 'job'
    ]
    for col in categorical_cols:                    #Ranking between categorical variables
        print(f"\nTop 5 values for {col}:")         #Clients plus présents, villes dominantes
        print(data[col].value_counts().head(5))     #professions majoritaires

    numeric_ranking = data[['qty', 'amount']].agg(['mean', 'sum', 'std']).T     #Ranking between numerical variables
    numeric_ranking = numeric_ranking.sort_values(by='sum', ascending=False)    #Most influent variable economicaly and volume
    print(numeric_ranking)

### **basic_info()** - Exploratory Data Analysis
**Purpose**: Understand the dataset structure and content.  
**Input**: DataFrame  
**Output**: Printed statistics (shape, columns, types, missing values, top categories, numeric rankings)  
**Use case**: Initial dataset inspection to identify data quality and feature importance  

**Code Approach**:  
Combines `.shape`, `.dtypes`, `.isna().sum()` for structure inspection, then uses `.value_counts().head(5)` for categorical ranking and `.agg(['mean', 'sum', 'std'])` for numeric ranking to identify key variables.

In [40]:
def preprocess_dates(data):
    """
    Function to preprocess date-related fields
    Input: a dataframe
    Output: the same dataframe with parsed dates, changing type and extracted year/month
    """
    data["invoice_date"] = pd.to_datetime(data["invoice_date"], format="%d/%m/%Y")  # Convert date string to datetime
    data["year"] = data["invoice_date"].dt.year                                      # Extract year
    data["month"] = data["invoice_date"].dt.month                                    # Extract month
    data["product_id"] = data["product_id"].astype(str)                              # Ensure product_id is string
    return data

### **preprocess_dates()** - Temporal Data Transformation
**Purpose**: Convert date strings to datetime format and extract temporal features.  
**Input**: DataFrame with string dates  
**Output**: DataFrame with datetime column + year/month columns  
**Use case**: Enables time-series analysis and temporal grouping for monthly sales trends  

**Code Approach**:  
Applies `pd.to_datetime()` with format string to parse dates correctly, then uses `.dt.year` and `.dt.month` accessors to extract temporal components. Creates new columns for easy grouping later.

Here we convert the `invoice_date` column into a real datetime format.  
We also create two new columns: `year` and `month`.

These will be useful later when we study trends in sales over time.

In [41]:
def sales_by_month(invoice):
    """
    Compute total monthly revenue based on 'amount'.
    """
    monthly_sales = invoice.groupby(["year", "month"])["amount"].sum().reset_index()
    monthly_sales["date"] = pd.to_datetime(
        monthly_sales["year"].astype(str) + "-" + monthly_sales["month"].astype(str) + "-01"
    )
    return monthly_sales

### **sales_by_month()** - Monthly Revenue Aggregation
**Purpose**: Calculate total sales per month for trend analysis.  
**Input**: Invoice DataFrame  
**Output**: DataFrame with monthly totals + date column  
**Use case**: Foundation for temporal visualization and forecasting (Indicator 3)  

**Code Approach**:  
Uses `.groupby(['year', 'month'])` to aggregate sales by time periods, then reconstructs a proper date column using `pd.to_datetime()` for cleaner visualizations.

This function calculates the total sales for each month.  
We group the data by year and month, then sum the amounts.

It gives us our first time-based indicator: how the store’s sales evolve over time.

In [42]:
def top_products(df, n=10):
    """
    Function to compute the top-N best-selling products
    Input: 
      df : dataframe containing at least 'product_id' and 'amount'
      n  : number of products to return (default = 10)
    Output: 
      a dataframe with the N products that generate the highest total amount
    """
    top = (
        df.groupby("product_id")["amount"]      # group by product and sum revenue
          .sum()
          .reset_index()                        # back to a flat dataframe
          .sort_values(by="amount", ascending=False)  # highest revenue first
          .head(n)                              # keep only top N
    )
    return top

### **top_products()** - **INDICATOR 1: Top Revenue Products**
**Purpose**: Identify best-selling products by total revenue (GroupBy + Aggregation).  
**Input**: DataFrame, number of top products (n=10)  
**Output**: Top N products with highest cumulative revenue  
**Use case**: Stock management, marketing focus, strategic planning  
**Visualization**: Bar chart (Product ID vs Revenue)  

**Code Approach**:  
Groups by `product_id`, sums amounts, sorts descending with `ascending=False`, and uses `.head(n)` to extract top N. Method chaining makes the pipeline clear and readable.

In [44]:
def pattern_mining_by_job(df, min_support=0.01, top_n=10):
    """
    Function to extract association rules by job
    Input:  
      df (with columns 'job', 'product_id', 'qty')
      min_support
      top_n
    Output: dataframe with the strongest association rules
    """

    basket = df.groupby(['job', 'product_id'])['qty'].sum().unstack().fillna(0)  # job–product matrix
    basket = basket > 0                                                           # convert quantities to booleans

    itemsets = fpgrowth(basket, min_support=min_support, use_colnames=True)       # frequent itemsets
    if itemsets.empty:                                                            # if nothing is frequent
        return pd.DataFrame({"message": ["No frequent itemsets found"]})          # return message instead

    rules = association_rules(itemsets, metric="lift", min_threshold=0)           # generate association rules
    if rules.empty:                                                               # if no rules are found
        return pd.DataFrame({"message": ["No association rules found"]})          # return message instead

    rules["score"] = rules["lift"] * rules["confidence"]                          # combined score = lift × confidence
    rules = rules.sort_values(by="score", ascending=False).head(top_n)            # keep only top_n best rules

    rules["antecedent_txt"] = rules["antecedents"].apply(lambda x: list(x)[0])    # convert antecedent set to text
    rules["consequent_txt"] = rules["consequents"].apply(lambda x: list(x)[0])    # convert consequent set to text

    def get_jobs_supporting_rule(row):                                            # helper: jobs that support a rule
        a = row["antecedent_txt"]                                                # product A (antecedent)
        b = row["consequent_txt"]                                                # product B (consequent)

        jobs_A = set(df[df["product_id"] == a]["job"])                           # jobs that bought A
        jobs_B = set(df[df["product_id"] == b]["job"])                           # jobs that bought B

        return sorted(jobs_A.intersection(jobs_B))                               # jobs that bought both A and B

    rules["jobs_supporting_rule"] = rules.apply(get_jobs_supporting_rule, axis=1) # add supporting jobs to each rule
    rules["num_jobs"] = rules["jobs_supporting_rule"].apply(len)

    return rules[[
        "antecedent_txt",                                                        # product on the left side of rule
        "consequent_txt",                                                        # product on the right side of rule
        "confidence",                                                            # confidence of the rule
        "lift",                                                                  # lift of the rule
        "score",                                                                 # combined score (lift × confidence)
        "jobs_supporting_rule",                                                  # list of jobs supporting the rule
        "num_jobs"
    ]]

### **pattern_mining_by_job()** - **INDICATOR 2: Product Association Rules**
**Purpose**: Discover product pairs bought together using FP-Growth + Association Rules (Frequent Pattern Mining).  
**Input**: DataFrame, min_support, top_n rules  
**Output**: Association rules with confidence, lift, and supporting job categories  
**Use case**: Cross-selling strategies, bundling, promotions  
**Visualization**: Scatter plot (Confidence vs Lift, sized by score)  

**Code Approach**:  
1. Creates job-product matrix using `.groupby().unstack()` and converts to boolean
2. Applies `fpgrowth()` to find frequent itemsets above min_support threshold
3. Uses `association_rules()` with lift metric to generate rules
4. Scores rules as `lift × confidence` and ranks top_n
5. Maps job support using set intersection: jobs that bought both A and B

In [45]:
from geopy.geocoders import Nominatim
import time

def spatial_analysis_by_city(
    df,
    city_col='city',
    amount_col='amount',
    geocode=False,
    top_n=15
):
    """
    Perform spatial analysis using city information:
    - aggregate invoice count and total revenue per city
    - optionally geocode only the top N cities
    """

    # Aggregation
    city_agg = (
        df
        .groupby(city_col)
        .agg(
            invoice_count=(city_col, 'count'),
            total_revenue=(amount_col, 'sum')
        )
        .reset_index()
        .sort_values(by='invoice_count', ascending=False)
    )

    # Keep only top N cities (important)
    city_agg = city_agg.head(top_n)

    # Optional geocoding
    if geocode:
        geolocator = Nominatim(user_agent="spatial_analysis_tp")
        latitudes = []
        longitudes = []

        for city in city_agg[city_col]:
            location = geolocator.geocode(f"{city}, France")
            if location:
                latitudes.append(location.latitude)
                longitudes.append(location.longitude)
            else:
                latitudes.append(None)
                longitudes.append(None)
            time.sleep(1)  # respect Nominatim policy

        city_agg['latitude'] = latitudes
        city_agg['longitude'] = longitudes

    return city_agg


### **spatial_analysis_by_city()** - **INDICATOR 4: Geographic Clustering**
**Purpose**: Analyze commercial activity by city + optional geocoding.  
**Input**: DataFrame, city column, amount column, geocode flag  
**Output**: Top N cities with invoice counts, revenue, and coordinates  
**Use case**: Logistics optimization, regional marketing, distribution strategy  
**Visualization**: Mapbox scatter (geographic position, circle size = invoices, color = revenue)  

**Code Approach**:  
Aggregates data using `.groupby().agg()` with multiple metrics (count, sum), sorts by invoice count, and if `geocode=True`, uses Nominatim API to convert city names to lat/lon coordinates. Rate limiting (1 sec delay) respects API policies.

In [46]:
def temporal_analysis(df, last_n_years=5, window_months=6, forecast_months=0):
    """
    Function for temporal analysis + optional Prophet forecasting
    Input:  
      df (invoice dataset)
      last_n_years
      window_months
      forecast_months
    Output: 
      monthly_df (clean monthly series)
      forecast_df (if enabled)
    """

    df = df.copy()                                                                  # work on a local copy

    max_year = df["invoice_date"].dt.year.max()                                     # most recent year in dataset
    min_year = max_year - last_n_years + 1                                          # oldest year to keep
    df = df[df["invoice_date"].dt.year >= min_year]                                 # filter selected years

    monthly = (                                                                     # compute monthly revenue
        df.groupby(df["invoice_date"].dt.to_period("M"))["amount"]                  
        .sum()
        .to_timestamp()
        .reset_index(name="amount")
        .rename(columns={"invoice_date": "date"})                                    # rename for clarity
    )

    monthly["trend"] = monthly["amount"].rolling(window=window_months).mean()        # rolling mean trend

    if forecast_months > 0:                                                          # forecasting enabled?
        prophet_df = monthly.rename(columns={"date": "ds", "amount": "y"})            # Prophet column format
        model = Prophet(yearly_seasonality=True)                                      # Prophet model
        model.fit(prophet_df[["ds", "y"]])                                            # train model
        future = model.make_future_dataframe(periods=forecast_months, freq="ME")      # extend timeline
        forecast = model.predict(future)                                              # generate forecast
    else:
        forecast = None                                                               # no forecasting

    return monthly, forecast                                                        

### **temporal_analysis()** - **INDICATOR 3: Time-Series Forecasting**
**Purpose**: Extract temporal trends and forecast future sales using Prophet.  
**Input**: DataFrame, time window, forecast horizon  
**Output**: Monthly aggregated data + Prophet forecast with confidence intervals  
**Use case**: Budget planning, inventory management, strategic forecasting  
**Visualization**: Line chart (actual sales, trend line, forecast + confidence zone)  

**Code Approach**:  
1. Filters data to last N years using year comparison
2. Groups by month using `.dt.to_period('M')` and aggregates revenue
3. Calculates rolling average trend using `.rolling(window=window_months).mean()`
4. If forecasting enabled: fits Prophet model, generates future dates, and predicts yhat + confidence intervals

In [47]:
# Main pipeline
def main():

    # --------------------------
    # 1. Load and preprocess data
    # --------------------------
    invoice = load_data("invoices.csv")
    df_cities = load_data("CitiesofFrance-VillesdeFrance.csv")

    cities = df_cities.iloc[:, 0].dropna().unique()
    invoice['city'] = np.random.choice(
        cities,
        size=len(invoice),
        replace=True
    )
    
    basic_info(invoice)
    invoice = preprocess_dates(invoice)

    # --------------------------
    # 2. Compute core indicators
    # --------------------------
    monthly_sales = sales_by_month(invoice)
    top10 = top_products(invoice)
    avg_basket = average_basket(invoice)

    # --------------------------
    # 3. Pattern mining (job-based rules)
    # --------------------------
    rules = pattern_mining_by_job(invoice, min_support=0.02, top_n=10)

    # --------------------------
    # 4. Temporal analysis + Prophet forecasting
    # --------------------------
    monthly, forecast = temporal_analysis(
    invoice,
    last_n_years=10,
    window_months=6,
    forecast_months=12
    )

    # --------------------------
    # 5. City spatial analysis (commercial activity)
    # --------------------------
    city_amount = spatial_analysis_by_city(invoice,geocode=True)

    print("Pipeline executed successfully.")

    # Return all outputs
    return {
        "invoice": invoice,
        "monthly_sales": monthly_sales,
        "top10": top10,
        "avg_basket": avg_basket,
        "rules": rules,
        "monthly": monthly,
        "forecast": forecast,
        "city_amount" : city_amount
    }

### **main()** - Pipeline Orchestration
**Purpose**: Execute the complete data analysis pipeline in sequence.  
**Steps**:
1. Load raw data + assign cities
2. Explore dataset structure
3. Extract 4 key indicators (top products, patterns, forecasts, geographic clusters)
4. Return all results as dictionary  
**Use case**: Central function called by Dash app to generate all visualizations  

**Code Approach**:  
Orchestrates all functions in correct order: load → preprocess → compute indicators → combine results into a dictionary. Uses `np.random.choice()` to assign random French cities (since data lacks real locations). Returns dict allows flexible access by Dash app.

In [48]:
from dash import Dash, dcc, html, dash_table
import dash_bootstrap_components as dbc
import plotly.express as px

#Main execution
if __name__ == "__main__":
    data = main()

# --------- Run your pipeline ---------
top10 = data["top10"]
rules = data["rules"]
monthly = data["monthly"]
forecast = data["forecast"]
city = data["city_amount"]

# ============================================
# 1. FIGURE Top 10 Products
# ============================================
fig_top10 = px.bar(
    top10,
    x="product_id",
    y="amount",
    title="Top 10 Products"
)

# ============================================
# 2. FIGURE Pattern Mining Rules
# ============================================
fig_rules = px.scatter(
    rules,
    x="confidence",
    y="lift",
    size="score",
    color="consequent_txt",
    hover_name="antecedent_txt",
    hover_data=["num_jobs"],
    title="Association Rules (Products Bought Together)"
)

# ============================================
# 3. FIGURE Temporal Analysis (monthly + trend + forecast)
# ============================================

fig_temp = px.line(
    monthly,
    x="date",
    y="amount",
    title="Monthly Sales (Last 10 Years)"
)
fig_temp.data[0].name = "Monthly Sales"
fig_temp.data[0].showlegend = True

# Add trend line
fig_temp.add_scatter(
    x=monthly["date"],
    y=monthly["trend"],
    mode="lines",
    name="Trend (Rolling Mean)",
    line=dict(color="orange", width=3)
)

# Add forecast
if forecast is not None:
    fig_temp.add_scatter(
        x=forecast["ds"],
        y=forecast["yhat"],
        mode="lines",
        name="Forecast",
        line=dict(color="green", width=2)
    )
    future = forecast[forecast["ds"] > monthly["date"].max()]

    fig_temp.add_scatter(
        x=list(future["ds"]) + list(future["ds"][::-1]),
        y=list(future["yhat_upper"]) + list(future["yhat_lower"][::-1]),
        fill="toself",
        fillcolor="rgba(0, 128, 0, 0.15)",
        line=dict(color="rgba(0,0,0,0)"),
        hoverinfo="skip",
        name="Confidence Interval"
    )

# ============================================
# 4. FIGURE City Clustering
# ============================================

fig_city = px.scatter_mapbox(
    city.head(10),
    lat='latitude',
    lon='longitude',
    size='invoice_count',          # taille des cercles
    color='total_revenue',         # couleur = revenu
    hover_name='city',
    size_max=20,
    zoom=5,
    title='Spatial Distribution of Invoice Activity by City',
    labels={
        'invoice_count': 'Number of Invoices',
        'total_revenue': 'Total Revenue'
    }
)

fig_city.update_layout(
    mapbox_style='carto-positron',
    margin=dict(r=0, t=40, l=0, b=0)
)

# ============================================
# DASH APP LAYOUT
# ============================================

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    
    html.H1("Invoices Dashboard", style={"textAlign": "center"}),

    html.H2("Top 10 Products"),
    dcc.Graph(figure=fig_top10),

    html.H2("Pattern Mining (Job-Based Rules)"),
    dcc.Graph(figure=fig_rules),

    html.H2("Temporal Analysis (Monthly + Trend + Forecast)"),
    dcc.Graph(figure=fig_temp),

    html.H2("City Clustering (Commercial Activity)"),
    dcc.Graph(figure=fig_city)
])

if __name__ == "__main__":
    app.run(debug=True)

Shape: (10000, 11)

Columns: ['first_name', 'last_name', 'email', 'product_id', 'qty', 'amount', 'invoice_date', 'address', 'city', 'stock_code', 'job']

Data types:
first_name       object
last_name        object
email            object
product_id        int64
qty               int64
amount          float64
invoice_date     object
address          object
city             object
stock_code        int64
job              object
dtype: object

Missing values per column:
first_name      0
last_name       0
email           0
product_id      0
qty             0
amount          0
invoice_date    0
address         0
city            0
stock_code      0
job             0
dtype: int64

Descriptive statistics:

Top 5 values for first_name:
first_name
David Williams        6
Daniel Johnson        5
Melissa Johnson       5
Michael Brown         5
Christopher Garcia    5
Name: count, dtype: int64

Top 5 values for city:
city
Rueil-la-Gadelière    4
Orus                  4
Mathonville           4
Homb

12:53:48 - cmdstanpy - INFO - Chain [1] start processing
12:53:48 - cmdstanpy - INFO - Chain [1] done processing


Pipeline executed successfully.


C:\Users\Aleksander\AppData\Local\Temp\ipykernel_17396\104563914.py:87: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

c:\Users\Aleksander\AppData\Local\Programs\Python\Python313\Lib\site-packages\plotly\express\_core.py:2530: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

c:\Users\Aleksander\AppData\Local\Programs\Python\Python313\Lib\site-packages\_plotly_utils\basevalidators.py:2640: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

c:\Users\Aleksander\AppData\Local\Programs\Python\Python313\Lib\site-packages\plotly\express\_core.py:2557: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

c:\Users\Aleksander\AppData\Local\

### **Dash Interactive Dashboard** - Final Presentation
**Purpose**: Create interactive web dashboard displaying all 4 key indicators.  
**Components**:
1. Bar chart: Top 10 products
2. Scatter plot: Association rules
3. Line chart: Temporal trends + forecast
4. Mapbox: Geographic distribution  
**Use case**: Real-time business intelligence, stakeholder communication, decision support  
**Run**: Execute this cell to launch the Dash server (http://127.0.0.1:8050/)  

**Code Approach**:  
Uses Plotly Express to create interactive figures, then wraps them in Dash HTML layout with Bootstrap styling. Scatter layers added dynamically (trend + forecast + CI) to temporal chart. Mapbox uses CARTO positron style for clean cartography. `app.run(debug=True)` launches local server with hot-reload enabled.

## Explanation of 4 Key Indicators

### **Top 10 Products** (Bar Chart - Blue)
**What is it?**  
The 10 products that generate the **highest revenue** for the store.

**Concrete Interpretation:**
- **Product 164** is the champion (~7000€) → our best-seller
- The 10 products are relatively balanced between 6000-7000€
- **Action**: Maintain sufficient stock on these 10 products

---

### **Pattern Mining - Association Rules** (Scatter Plot - Colors)
**What is it?**  
The **pairs of products bought together** by customers in the same profession.

**Graph Axes**:
- **Confidence (X-axis)**: Probability that if customer buys A, they also buy B (27%-31%)
- **Lift (Y-axis)**: How many times more likely than random (1.7x to 2.0x)
- **Point Size**: Combined rule strength (score = lift × confidence)
- **Color**: Consequent product (product B purchased second)

**Concrete Interpretation**:
- Point in **top-right** = best rule (high confidence and lift)
- **Example**: If a customer buys Product 189, there's a 30% chance they'll also buy Product X
- **Action**: Propose these pairs as bundles or promotions

---

### **Temporal Analysis** (Line Chart - Blue/Orange/Green)
**What is it?**  
The **sales evolution** over 10 years + future predictions.

**3 Curves**:
1. **Blue** (Monthly Sales): Actual monthly sales (noisy, volatile)
2. **Orange** (Trend): Smooth trend (6-month rolling average) → stable general direction
3. **Green** (Forecast): Prophet predictions for next 12 months
4. **Green Zone**: Confidence interval (95%) → uncertainty increases with time

**Concrete Interpretation**:
- Stable trend → no major growth/decline
- Confidence zone widens towards future (uncertainty increases)
- **Action**: Plan 12 months of inventory based on Trend

---

### **City Clustering - Spatial Distribution** (Map)
**What is it?**  
The **location of cities** based on their commercial activity.

**Map Elements**:
- **Position**: True geographic location (latitude/longitude)
- **Circle Size**: Number of invoices per city (larger = more purchases)
- **Color**: Total revenue generated (yellow = richer, blue = less)

**Concrete Interpretation**:
- **Large yellow circles**: VIP cities (many purchases + high amounts)
- **Blue circles**: Emerging or niche cities
- **Action**: Invest in logistics for major cities (Paris, Lyon)

---

### **Summary - What decisions should we take?**

| Indicator | Commercial Decision |
|-----------|---------------------|
| **Top 10 products** | Stock management, marketing campaigns |
| **Association Rules** | Bundles, cross-selling strategies |
| **Temporal Analysis** | Budget planning, annual forecasts |
| **City Clustering** | Distribution, logistics infrastructure |